In [6]:
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
import list_initial_conditions as ic
import list_ibvp_solution as ibvp
import list_swwe_function as swe
from time import time
from flowtype import flowtype_function

start=time()
print("Start convergence test") 
# set up nature of the simulation, with or without manufactured solution:
# 1. manufactured_solution = ibvp.linearSWWE_mms
# 2. manufactured_solution = False
mms = ibvp.linearSWWE_mms
number_of_iteration = 9

SAT_function = swe.linearised_SWWE_SAT_terms
RHS = swe.linearswwe_RHS

# set 'generated_wave = False' if manufactured_solution is not False
generated_wave = False
# generated_wave = ibvp.zero_wave
# generated_wave = ibvp.gaussian_wave_1
# generated_wave = ibvp.step_function_wave


# set up parameters
cfl = 0.25
H = 1.0
g = 9.81
c = np.sqrt(g*H)

# coefficient may vary 
U_coeff = 0.5
alpha_coeff = 0.0

U = U_coeff*c
alpha = alpha_coeff*(np.abs(U)+c)

flowtype = flowtype_function(U,c)

error_h = np.zeros(number_of_iteration)
error_u = np.zeros(number_of_iteration)

now = time()
for N0 in range(number_of_iteration):
        # set up the domain 
        # x0, xN = 0, 4
        x0, xN = 0,(np.abs(U)+np.sqrt(g*H))*5
        N = 2**(4+N0)
        print(f"N0: {N0}. Number of grid points: {N}")
        dx = (xN-x0)/(N-1)  
        x = np.linspace(x0,xN,N)

        # set up time parameters
        dt = cfl*dx/(np.abs(U) + np.sqrt(g*H))      # (cfl * dx)/max speed
        sim_time = np.pi*0.2

        # set up supportive matrix 
        Q, A, P_inv = swe.linearised_SWWE_matrix_supportive(N, dx)

        # list constants
        constants = x,H,U,g,c,alpha,flowtype,Q,A,P_inv, \
                RHS, SAT_function, \
                mms, generated_wave 

        # x,H,U,g,c,alpha,flowtype,Q,A,P_inv,RHS_function, SAT,manufactured_solution, analytical_solution = constants

        # solving the IBVP 

        # set up initial conditions
        q, _ = ibvp.linearSWWE_mms(x,0,H,U,g)
        q_numerical = copy(q)
        q_analytical = copy(q)

        q_numerical = swe.numerical_solution(q_numerical,sim_time,dt,constants)
        q_analytical = swe.analytical_solution_mms(x,sim_time,dt,constants)

        now = time()
        
        # error calculation
        q_numerical = np.array(q_numerical[-1])
        q_analytical = np.array(q_analytical[-1])

        error_h[N0] = np.linalg.norm(q_numerical[0] - q_analytical[0])*np.sqrt(dx)
        error_u[N0] = np.linalg.norm(q_numerical[1] - q_analytical[1])*np.sqrt(dx)

error_rate_h = np.log2(error_h[:-1]/error_h[1:])
error_rate_u = np.log2(error_h[:-1]/error_h[1:])


end = time()

print(f"Number of iteration: {number_of_iteration}, {2**(6)} to  {2**(6+N0)}, dx: {dx}")
print(f"simulation time: {np.round(sim_time,4)} s, dt: {np.round(dt,4)} s")
print(f"U*sqrt(gH): {U_coeff}, alpha*(np.abs(U)+c): {alpha_coeff}")
print(f"U: {U}, alpha: {alpha}")
print(f"flowtype: {flowtype}")
print() 
print(f"Time elapsed: {np.round(end-start,4)} seconds")
print() 
print(f"Error in h: {error_rate_h}")
print(f"Error in u: {error_rate_u}")


# plt.plot(x,q[0], label=0)
# for i in range (int(sim_time//dt)):
#     q0 = q_numerical[i+1]
#     h, u = q0 
#     plt.plot(x,h,label=i)
#     plt.legend()
# plt.show()
# %%
 

N0: 8. Number of grid points: 4096
Number of iteration: 9, 64 to  16384, dx: 0.00573643214775305
simulation time: 0.6283 s, dt: 0.0003 s
U*sqrt(gH): 0.5, alpha*(np.abs(U)+c): 0.0
U: 1.5660459763365826, alpha: 0.0
flowtype: subcritical

Time elapsed: 32.4013 seconds

Error in h: [1.87355162 2.00387411 2.02559757 2.0081783  2.01266864 2.00190627
 2.00322697 2.00050183]
Error in u: [1.87355162 2.00387411 2.02559757 2.0081783  2.01266864 2.00190627
 2.00322697 2.00050183]


In [7]:
error_rate_h

array([1.87355162, 2.00387411, 2.02559757, 2.0081783 , 2.01266864,
       2.00190627, 2.00322697, 2.00050183])